In [283]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
import plotly.graph_objects as go


In [284]:
data = yf.download("CL=F", period="6mo")
# data = yf.download("CAD=X", period="6mo")

# Keep only the Close column and rename it
df = data[["Close"]].rename(columns={"Close": "CAD"}).reset_index().dropna()

# Ensure no multi-level columns remain
df.columns = ["Date", "CAD"]
df.set_index('Date', inplace=True)


C:\Users\minhd\AppData\Local\Temp\ipykernel_11516\2368436839.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [285]:
windows = {
    "1w": 5,
    "2w": 10,
    "1m": 30
}

for label, win in windows.items():
    df[f"rolling_mean_{label}"] = df["CAD"].rolling(win).mean()
    df[f"rolling_std_{label}"]  = df["CAD"].rolling(win).std()
    df[f"zscore_{label}"] = (df["CAD"] - df[f"rolling_mean_{label}"]) / df[f"rolling_std_{label}"]

# Drop rows where any z-score is NaN (start of series)
df = df.dropna(subset=["zscore_1w", "zscore_2w", "zscore_1m"])

z_thresh = 0.8

cond_1w_2w = (df["zscore_1w"].abs() <= z_thresh) & (df["zscore_2w"].abs() <= z_thresh)
cond_1m    = (df["zscore_1m"].abs() <= z_thresh)

df["ou_flag"] = np.where(cond_1w_2w | cond_1m, 1, 0)



In [286]:
df[['CAD']]

,CAD
Date,
2025-06-30,65.110001
2025-07-01,65.449997
2025-07-02,67.449997
2025-07-03,67.000000
2025-07-04,66.500000
...,...
2025-11-10,60.130001
2025-11-11,61.040001
2025-11-12,58.490002


In [287]:
regions = []
current_flag = None
start = None

for t, row in df.iterrows():
    if current_flag is None:
        # start first region
        current_flag = row["ou_flag"]
        start = t
    elif row["ou_flag"] != current_flag:
        # region ended
        regions.append((start, t, current_flag))
        start = t
        current_flag = row["ou_flag"]

# last region
regions.append((start, df.index[-1], current_flag))

# 5. Plot
fig = go.Figure()

# Main price line
fig.add_trace(go.Scatter(
    x=df.index,
    y=df["CAD"],
    mode="lines",
    name="CAD Spot"
))

# Shading OU vs non-OU
for (x0, x1, flag) in regions:
    if flag:   # OU mean-reverting region
        color = "green"
        opacity = 0.2
        # name = "OU regime"
    else:      # non-OU region
        color = "red"
        opacity = 0.2
        # name = "non-OU regime"

    fig.add_vrect(
        x0=x0, x1=x1,
        fillcolor=color,
        opacity=opacity,
        line_width=0,
        # annotation_text=name,
        # annotation_position="top left",
        # annotation_font_size=10
    )

fig.update_layout(
    title="CAD Mean Reversion vs Non-OU Highlighted Zones",
    xaxis_title="Date",
    yaxis_title="CAD Spot",
)

fig.show()